# Data cleaning

## Joining datasets for POS tagging

In [4]:
import numpy as np
import pandas as pd

# read datasets
# df1 = pd.read_csv('../raw/CEO Nov1_30 2022 CLEAN FINAL.csv')
df2 = pd.read_csv('../raw/CEO Oct 1_31 2022 CLEAN FINAL.csv')
df3 = pd.read_csv('../raw/CEO Sept 10_30 2023 CLEAN FINAL.csv')

df4 = pd.read_csv('../raw/LEO Aug 28 - Nov 15 2024 LABELS.csv')
df5 = pd.read_csv('../raw/State EO ALL data NO LABELS Aug 28 _ Dec 13 2024.csv')
df6 = pd.read_csv('../raw/State EO Coded Jan 22 2025 export .csv')

In [8]:
df2.head()

,PostId,OriginalLink,PublishedAt,Platform,ChannelName,ChannelId,LikesCount,SharesCount,CommentsCount,Candidate Trainings,...,THEME: Pre-Election procedures,THEME: Post-election procedures,THEME: Voter Registration,THEME: Voting Methods,THEME: Election Ready,THEME: Motivational/GOTV,THEME: Non-election related,Vote Centers,BOOKMARK,Notes
0,60533003,https://twitter.com/temp/status/15756322767584...,2022-09-29 23:43:03 UTC,Twitter,Nevada Elections,8891724,0,0,1,0,...,0,0,0,0,0,1,0,0,0,This post is about AB121 legislation passed t...
1,58338631,https://twitter.com/CASOSVote/status/157092555...,2022-09-17 00:00:13 UTC,Twitter,CA SOS Vote,8403253,16,9,2,0,...,0,0,0,0,1,0,0,0,0,Voters information guide (VIG)
2,60587077,https://twitter.com/temp/status/15744208053455...,2022-09-26 15:29:05 UTC,Twitter,Illinois SBE,8891698,12,4,1,0,...,0,0,0,0,0,0,0,0,0,Transparency
3,64634041,https://twitter.com/COSecofState/status/158249...,2022-10-18 22:25:27 UTC,Twitter,Colorado Sec. of State,8889067,13,8,1,0,...,0,0,0,0,1,0,0,0,1,Zero rides initiatives to help voters get to...
4,72439269,https://www.facebook.com/100064597335230/posts...,2022-10-27 20:00:03 UTC,Facebook,Colorado Secretary of State's Office,8894173,49,6,22,0,...,0,0,0,0,0,1,0,0,1,zero fair transit


In [6]:
# rename from df9 post_body_text to Description
df5 = df5.rename(columns={'post_body_text': 'Description'})

In [7]:
# For df1, df2, df3: Join Description and Caption (and AudioTranscript for df2 and df3)
# df1['Combined_text'] = df1['Description'].fillna('') + ' ' + df1['Caption'].fillna('')
df2['Combined_text'] = df2['Description'].fillna('') + ' ' + df2['Caption'].fillna('') + ' ' + df2['AudioTranscript'].fillna('')
df3['Combined_text'] = df3['Description'].fillna('') + ' ' + df3['Caption'].fillna('') + ' ' + df3['AudioTranscript'].fillna('')

# For df9: Join Description, GoogleAudioText, and EmbeddedContentText
df5['Combined_text'] = df5['Description'].fillna('') + ' ' + df5['GoogleAudioText'].fillna('') + ' ' + df5['EmbeddedContentText'].fillna('')

# For df8 and df10: Join Description, Caption, AudioTranscript, and EmbeddedContentText
df4['Combined_text'] = df4['Description'].fillna('') + ' ' + df4['Caption'].fillna('') + ' ' + df4['AudioTranscript'].fillna('') + ' ' + df4['EmbeddedContentText'].fillna('')
df6['Combined_text'] = df6['Description'].fillna('') + ' ' + df6['Caption'].fillna('') + ' ' + df6['AudioTranscript'].fillna('') + ' ' + df6['EmbeddedContentText'].fillna('')

KeyError: 'Description'

In [ ]:
# For each, keep only: PostId, EmbeddedContentText
# df1 = df1[['PostId', 'Combined_text']]
df2 = df2[['PostId', 'Combined_text', 'Platform', 'ChannelName']]
df3 = df3[['PostId', 'Combined_text', 'Platform', 'ChannelName']]
df4 = df4[['PostId', 'Combined_text', 'Platform', 'ChannelName']]
df5 = df5[['PostId', 'Combined_text', 'Platform', 'ChannelName']]
df6 = df6[['PostId', 'Combined_text', 'Platform', 'ChannelName']]

# Concatenate all DataFrames vertically
combined_df = pd.concat([df2, df3, df4, df5, df6], axis=0)

In [ ]:
# size of combined_df
print(f"Size of combined_df: {combined_df.shape[0]} rows")

Size of combined_df: 57988 rows


In [ ]:
# Step 1: Count duplicates for each PostId
duplicate_counts = combined_df.groupby('PostId').size().reset_index(name='Count')

# Step 2: Filter only the PostIds that have duplicates
duplicates = duplicate_counts[duplicate_counts['Count'] > 1]

# Step 3: Print the count of duplicates
print(f"Total duplicates removed: {len(duplicates)}")
print("\nDetails of duplicates:")
print(duplicates)

# Step 4: Remove duplicates (keep the first occurrence of each PostId)
deduplicated_combined_df = combined_df.drop_duplicates(subset='PostId', keep='first')

Total duplicates removed: 5880

Details of duplicates:
          PostId  Count
26944  303540827      2
26946  303540849      2
26948  303546389      2
26959  303649244      2
27000  303750710      2
...          ...    ...
52005  410958078      2
52007  411527280      2
52013  413182773      2
52016  413303947      2
52066  417003896      2

[5880 rows x 2 columns]


In [ ]:
# size of deduplicated_combined_df
print(f"Size of deduplicated_combined_df: {deduplicated_combined_df.shape[0]} rows")

Size of deduplicated_combined_df: 52108 rows


In [ ]:
# head
deduplicated_combined_df.head()

,PostId,Combined_text
0,74414837,We hope everyone has a safe and Happy Hallowee...
1,74420801,Oconee County has the best Elections staff and...
2,74420802,🇺🇸Keep on voting Young County🇺🇸 Let’s try and ...
3,74420805,"Early Voting turnout for Monday, October 31, 2..."
4,74411274,Happy Halloween from the Clerk-Recorders Office!


In [ ]:
# Save the combined DataFrame to a CSV file
deduplicated_combined_df.to_csv('../clean/pos_tagging_data.csv', index=False)